# Home Credit Default Risk 2018

__Warning!__ This kernel cannot run on Kaggle: not enough memory. I run everything on Google Datalab with 8 core highmem cpu

Based on the following kernel: 

- https://www.kaggle.com/aantonova/preliminary-analysis-of-application-dataset/notebook

## Import necessary modules

In [28]:
import sys
import google.datalab.storage as storage
from io import BytesIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [29]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier
from scipy.stats import ranksums
from bayes_opt import BayesianOptimization

## Aggregating datasets

### Service functions

In [34]:
#Reduce the storage used by the dataset
def reduce_mem_usage(data, verbose = True):
    """
    data: the original dataset
    verbose: if true, show the information of the data set after memory usage reduced
    
    The function returns a copy of the original dataset which uses less memory
    """
    start_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for col in data.columns:
        col_type = data[col].dtype
        
        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            #assign integer type based on its magnitude
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            #assign floating number type based on its magnitude
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)

    end_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return data

In [36]:
#for each categorical column, transform every category to an integer.
def cat_to_num(data, nan_as_category = True):
    """
    data : original data or data with reduced memory usage
    nan_as_category : Default is true, it will treat missing value as a new category.
    
    The function 
    """
    categorical_columns = [col for col in data.columns \
                        if not pd.api.types.is_numeric_dtype(data[col].dtype)]
    for c in categorical_columns:
        data.loc[:,c] = data.loc[:,c].astype("category")
        #fill na with categorical value
        if sum(pd.isna(data.loc[:,c])) > 0 and nan_as_category:
        data.loc[:,c] = data.loc[:,c].cat.add_categories("NaN").fillna("NaN")
        #replace category with number
        data.loc[:,c] = data.loc[:,c].cat.codes
    gc.collect()
    return data

In [37]:
def add_corr_col(data):
    """
    The function adds correlation columns to the copy of the original dataset, 
    each correlation column is the multiplication of two selected features. And then return the 
    new dataset
    """
    feats = [f for f in data.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    for i in range(len(feats)):
    for j in range(i, len(feats)):
        #The name of the correlation column is the combination of the names of the two selected features
        name1 = feats[i]
        name2 = feats[j]
        new_name = name1 + "_" + name2
        data[new_name] = data[name1].multiply(data[name2])
    gc.collect()
    return data

In [38]:
#Summarize the new features on client level
def summarize_data(data, data_name, group_name, length_name):
    """
    data : original data
    data_name : name of the dataset
    group_name : the column for client id
    length_name : the name of the column that summarize the length of a client's history
    The function aggregate the data on client level and return the aggregated dataset
    """
    #Aggregate data on client level
    data = cat_to_num(data)
    data = add_corr_col(data)
    aggregations = {}
    data_agg = data.groupby(group_name).mean()
    #Define the names of the aggregate data
    new_col = [data_name + col for col in data_agg.columns]
    data_agg.columns = new_col
    data_agg[length_name] = data.groupby(group_name).size()
    del data
    gc.collect()
    return data_agg

### Aggregating functions

In [39]:
def application_train_test(nan_as_category = True):
    """
    The function aggregates application_train.csv and application_test.csv together
    and return the aggregated dataset
    """
    # Read data and merge
    temp_data1 = storage.Object('firstproject-214518', 'dataset/application_train.csv').read_stream()
    df_train = pd.read_csv(BytesIO(temp_data1))
    print("The dataset application_train has the shape {}".format(df_train.shape))
    temp_data2 = storage.Object('firstproject-214518', 'dataset/application_test.csv').read_stream()
    df_test = pd.read_csv(BytesIO(temp_data2))
    df = pd.concat([df_train, df_test], axis = 0, ignore_index = True)
    del df_train, df_test, temp_data1, temp_data2
    gc.collect()
    
    # Remove some rows with values not present in test set
    df.drop(df[df['CODE_GENDER'] == 'XNA'].index, inplace = True)
    df.drop(df[df['NAME_INCOME_TYPE'] == 'Maternity leave'].index, inplace = True)
    df.drop(df[df['NAME_FAMILY_STATUS'] == 'Unknown'].index, inplace = True)
    
    # Remove some empty features
    df.drop(['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 
            'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 
            'FLAG_DOCUMENT_21'], axis = 1, inplace = True)
    
    # Replace some outliers
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)
    df.loc[df['OWN_CAR_AGE'] > 80, 'OWN_CAR_AGE'] = np.nan
    df.loc[df['REGION_RATING_CLIENT_W_CITY'] < 0, 'REGION_RATING_CLIENT_W_CITY'] = np.nan
    df.loc[df['AMT_INCOME_TOTAL'] > 1e8, 'AMT_INCOME_TOTAL'] = np.nan
    df.loc[df['AMT_REQ_CREDIT_BUREAU_QRT'] > 10, 'AMT_REQ_CREDIT_BUREAU_QRT'] = np.nan
    df.loc[df['OBS_30_CNT_SOCIAL_CIRCLE'] > 40, 'OBS_30_CNT_SOCIAL_CIRCLE'] = np.nan
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], _ = pd.factorize(df[bin_feature])
        
    # transform all categories in the categorical column as integers
    df = cat_to_num(df, nan_as_category)
    
    # Some new features
    df['app missing'] = df.isnull().sum(axis = 1).values
    df['app AMT_CREDIT - AMT_GOODS_PRICE'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']
    df['app AMT_CREDIT / AMT_GOODS_PRICE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['app AMT_CREDIT / AMT_ANNUITY'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['app AMT_CREDIT / AMT_INCOME_TOTAL'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    
    df['app AMT_INCOME_TOTAL / 12 - AMT_ANNUITY'] = df['AMT_INCOME_TOTAL'] / 12. - df['AMT_ANNUITY']
    df['app AMT_INCOME_TOTAL / AMT_ANNUITY'] = df['AMT_INCOME_TOTAL'] / df['AMT_ANNUITY']
    df['app AMT_INCOME_TOTAL - AMT_GOODS_PRICE'] = df['AMT_INCOME_TOTAL'] - df['AMT_GOODS_PRICE']
    df['app AMT_INCOME_TOTAL / CNT_FAM_MEMBERS'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['app AMT_INCOME_TOTAL / CNT_CHILDREN'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
    
    df['app most popular AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
                        .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
    df['app popular AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
                        .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
    
    df['app OWN_CAR_AGE / DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['app OWN_CAR_AGE / DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    
    df['app DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    df['app DAYS_LAST_PHONE_CHANGE / DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['app DAYS_EMPLOYED - DAYS_BIRTH'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
    df['app DAYS_EMPLOYED / DAYS_BIRTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    df['app CNT_CHILDREN / CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
    gc.collect()
    return reduce_mem_usage(df)

In [40]:
def bureau_and_balance(nan_as_category = True):
    """
    This function aggregates datasets related with bureau and return the aggregated dataset 
    """
    #Read in the data and reduce its memory usage
    temp_data1 = storage.Object('firstproject-214518', 'dataset/bureau_balance.csv').read_stream()
    df_bureau_b = reduce_mem_usage(pd.read_csv(BytesIO(temp_data1)), verbose = False)
    del temp_data1
    print("The dataset bureau_balance has the shape {}".format(df_bureau_b.shape))
    #summarize all the information on client level
    df_bureau_b_agg = summarize_data(df_bureau_b, "bureau_balance","SK_ID_BUREAU", "balance_hist_length")
    del df_bureau_b
    gc.collect()
    
    temp_data2 = storage.Object('firstproject-214518', 'dataset/bureau.csv').read_stream()
    df_bureau = reduce_mem_usage(pd.read_csv(BytesIO(temp_data2)),verbose = False)
    del temp_data2
    print("The dataset bureau has the shape {}".format(df_bureau.shape))
    # Replace\remove some outliers in bureau set
    df_bureau.loc[df_bureau['AMT_ANNUITY'] > .8e8, 'AMT_ANNUITY'] = np.nan
    df_bureau.loc[df_bureau['AMT_CREDIT_SUM'] > 3e8, 'AMT_CREDIT_SUM'] = np.nan
    df_bureau.loc[df_bureau['AMT_CREDIT_SUM_DEBT'] > 1e8, 'AMT_CREDIT_SUM_DEBT'] = np.nan
    df_bureau.loc[df_bureau['AMT_CREDIT_MAX_OVERDUE'] > .8e8, 'AMT_CREDIT_MAX_OVERDUE'] = np.nan
    df_bureau.loc[df_bureau['DAYS_ENDDATE_FACT'] < -10000, 'DAYS_ENDDATE_FACT'] = np.nan
    df_bureau.loc[(df_bureau['DAYS_CREDIT_UPDATE'] > 0) | (df_bureau['DAYS_CREDIT_UPDATE'] < -40000), 'DAYS_CREDIT_UPDATE'] = np.nan
    df_bureau.loc[df_bureau['DAYS_CREDIT_ENDDATE'] < -10000, 'DAYS_CREDIT_ENDDATE'] = np.nan
    df_bureau.drop(df_bureau[df_bureau['DAYS_ENDDATE_FACT'] < df_bureau['DAYS_CREDIT']].index, inplace = True)
    
    
    # Bureau balance: merge with bureau.csv
    df_bureau = df_bureau.join(df_bureau_b_agg, how = 'left', on = 'SK_ID_BUREAU')
    df_bureau.drop('SK_ID_BUREAU', axis = 1, inplace = True)
    del df_bureau_b_agg
    gc.collect()
    
    df_bureau_agg = summarize_data(df_bureau,"bureau", "SK_ID_CURR", "bureau_hist_length")
    del df_bureau
    gc.collect()
    return reduce_mem_usage(df_bureau_agg)

In [41]:
def previous_application(nan_as_category = True):
    """
    The function aggregates the credit history datasets together and return the aggregated dataset
    """
    temp_data = storage.Object('firstproject-214518', 'dataset/previous_application.csv').read_stream()
    df_prev = reduce_mem_usage(pd.read_csv(BytesIO(temp_data)))
    del temp_data
    print("The dataset previous_application has the shape {}".format(df_prev.shape))
    # Replace some outliers
    df_prev.loc[df_prev['AMT_CREDIT'] > 6000000, 'AMT_CREDIT'] = np.nan
    df_prev.loc[df_prev['SELLERPLACE_AREA'] > 3500000, 'SELLERPLACE_AREA'] = np.nan
    df_prev[['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 
             'DAYS_LAST_DUE', 'DAYS_TERMINATION']].replace(365243, np.nan, inplace = True)
    # Some new features
    df_prev['prev missing'] = df_prev.isnull().sum(axis = 1).values
    df_prev['prev AMT_APPLICATION / AMT_CREDIT'] = df_prev['AMT_APPLICATION'] / df_prev['AMT_CREDIT']
    df_prev['prev AMT_APPLICATION - AMT_CREDIT'] = df_prev['AMT_APPLICATION'] - df_prev['AMT_CREDIT']
    df_prev['prev AMT_APPLICATION - AMT_GOODS_PRICE'] = df_prev['AMT_APPLICATION'] - df_prev['AMT_GOODS_PRICE']
    df_prev['prev AMT_GOODS_PRICE - AMT_CREDIT'] = df_prev['AMT_GOODS_PRICE'] - df_prev['AMT_CREDIT']
    df_prev['prev DAYS_FIRST_DRAWING - DAYS_FIRST_DUE'] = df_prev['DAYS_FIRST_DRAWING'] - df_prev['DAYS_FIRST_DUE']
    df_prev['prev DAYS_TERMINATION less -500'] = (df_prev['DAYS_TERMINATION'] < -500).astype(int)
    #summarize the data on client level
    df_prev_agg = summarize_data(df_prev,"previous", "SK_ID_CURR", "prev_app_hist_length")
    del df_prev
    gc.collect()
    return reduce_mem_usage(df_prev_agg)

In [42]:
def pos_cash(nan_as_category = True):
    """
    The dataset aggregate the POS_CASH_balance.csv dataset on client level and return the aggregated dataset
    """
    temp_data = storage.Object('firstproject-214518', 'dataset/POS_CASH_balance.csv').read_stream()
    df_pos = reduce_mem_usage(pd.read_csv(BytesIO(temp_data)))
    del temp_data
    print("The dataset pos_cash_balance has the shape {}".format(df_pos.shape))
    # Replace some outliers
    df_pos.loc[df_pos['CNT_INSTALMENT_FUTURE'] > 60, 'CNT_INSTALMENT_FUTURE'] = np.nan
    
    # some new features
    df_pos['pos CNT_INSTALMENT more CNT_INSTALMENT_FUTURE'] = \
                    (df_pos['CNT_INSTALMENT'] > df_pos['CNT_INSTALMENT_FUTURE']).astype(int)
    
    # summarize the data on client level
    df_pos_agg = summarize_data(df_pos,"pos", "SK_ID_CURR", "pos_hist_length")
    del df_pos
    gc.collect()
    return reduce_mem_usage(df_pos_agg)

In [43]:
def installments_payments(nan_as_category = True):
    """
    The function aggregate the installments_payments.csv dataset, and return the aggregated one
    """
    temp_data = storage.Object('firstproject-214518', 'dataset/installments_payments.csv').read_stream()
    df_ins = reduce_mem_usage(pd.read_csv(BytesIO(temp_data)))
    del temp_data
    print("The datset installments_payments has the shape {}".format(df_ins.shape))
    # Replace some outliers
    df_ins.loc[df_ins['NUM_INSTALMENT_VERSION'] > 70, 'NUM_INSTALMENT_VERSION'] = np.nan
    df_ins.loc[df_ins['DAYS_ENTRY_PAYMENT'] < -4000, 'DAYS_ENTRY_PAYMENT'] = np.nan
    
    # Some new features
    df_ins['ins DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT'] = df_ins['DAYS_ENTRY_PAYMENT'] - df_ins['DAYS_INSTALMENT']
    df_ins['ins NUM_INSTALMENT_NUMBER_100'] = (df_ins['NUM_INSTALMENT_NUMBER'] == 100).astype(int)
    df_ins['ins DAYS_INSTALMENT more NUM_INSTALMENT_NUMBER'] = (df_ins['DAYS_INSTALMENT'] > df_ins['NUM_INSTALMENT_NUMBER'] * 50 / 3 - 11500 / 3).astype(int)
    df_ins['ins AMT_INSTALMENT - AMT_PAYMENT'] = df_ins['AMT_INSTALMENT'] - df_ins['AMT_PAYMENT']
    df_ins['ins AMT_PAYMENT / AMT_INSTALMENT'] = df_ins['AMT_PAYMENT'] / df_ins['AMT_INSTALMENT']
    #summarize the data on client level
    df_ins_agg = summarize_data(df_ins,"ins", "SK_ID_CURR", "ins_hist_length") 
    del df_ins
    gc.collect()  
    return reduce_mem_usage(df_ins_agg)

In [44]:
def credit_card_balance(nan_as_category = True):
    """
    Aggregate the credit card data and return it
    """
    temp_data = storage.Object('firstproject-214518', 'dataset/credit_card_balance.csv').read_stream()
    df_card = reduce_mem_usage(pd.read_csv(BytesIO(temp_data)))
    print("The dataset credit_card_balance has the shape {}".format(df_card.shape))
    del temp_data
    # Replace some outliers
    df_card.loc[df_card['AMT_PAYMENT_CURRENT'] > 4000000, 'AMT_PAYMENT_CURRENT'] = np.nan
    df_card.loc[df_card['AMT_CREDIT_LIMIT_ACTUAL'] > 1000000, 'AMT_CREDIT_LIMIT_ACTUAL'] = np.nan
     # Some new features
    df_card['card missing'] = df_card.isnull().sum(axis = 1).values
    df_card['card SK_DPD - MONTHS_BALANCE'] = df_card['SK_DPD'] - df_card['MONTHS_BALANCE']
    df_card['card SK_DPD_DEF - MONTHS_BALANCE'] = df_card['SK_DPD_DEF'] - df_card['MONTHS_BALANCE']
    df_card['card SK_DPD - SK_DPD_DEF'] = df_card['SK_DPD'] - df_card['SK_DPD_DEF']
  
    df_card['card AMT_TOTAL_RECEIVABLE - AMT_RECIVABLE'] = df_card['AMT_TOTAL_RECEIVABLE'] - df_card['AMT_RECIVABLE']
    df_card['card AMT_TOTAL_RECEIVABLE - AMT_RECEIVABLE_PRINCIPAL'] = df_card['AMT_TOTAL_RECEIVABLE'] - df_card['AMT_RECEIVABLE_PRINCIPAL']

    df_card['card AMT_RECIVABLE - AMT_RECEIVABLE_PRINCIPAL'] = df_card['AMT_RECIVABLE'] - df_card['AMT_RECEIVABLE_PRINCIPAL']

    df_card['card AMT_BALANCE - AMT_RECIVABLE'] = df_card['AMT_BALANCE'] - df_card['AMT_RECIVABLE']
    df_card['card AMT_BALANCE - AMT_RECEIVABLE_PRINCIPAL'] = df_card['AMT_BALANCE'] - df_card['AMT_RECEIVABLE_PRINCIPAL']
    df_card['card AMT_BALANCE - AMT_TOTAL_RECEIVABLE'] = df_card['AMT_BALANCE'] - df_card['AMT_TOTAL_RECEIVABLE']

    df_card['card AMT_DRAWINGS_CURRENT - AMT_DRAWINGS_ATM_CURRENT'] = df_card['AMT_DRAWINGS_CURRENT'] - df_card['AMT_DRAWINGS_ATM_CURRENT']
    df_card['card AMT_DRAWINGS_CURRENT - AMT_DRAWINGS_OTHER_CURRENT'] = df_card['AMT_DRAWINGS_CURRENT'] - df_card['AMT_DRAWINGS_OTHER_CURRENT']
    df_card['card AMT_DRAWINGS_CURRENT - AMT_DRAWINGS_POS_CURRENT'] = df_card['AMT_DRAWINGS_CURRENT'] - df_card['AMT_DRAWINGS_POS_CURRENT']
    
    df_card_agg = summarize_data(df_card,"card", "SK_ID_CURR", "credit_hist_length")
    
    del df_card
    gc.collect()
    
    return reduce_mem_usage(df_card_agg)

In [45]:
def aggregate():
    """
    This function merge(left join) all datasets together and return 
    """
    warnings.simplefilter(action = 'ignore')
    
    print('-' * 20)
    print('1: application train & test (', time.ctime(), ')')
    print('-' * 20)
    df = application_train_test()
    print('     DF shape:', df.shape)   
    print('-' * 20)
    print('2: bureau & balance (', time.ctime(), ')')
    print('-' * 20)
    bureau = bureau_and_balance()
    df = df.join(bureau, how = 'left', on = 'SK_ID_CURR')
    print('     DF shape:', df.shape)
    del bureau
    gc.collect()
    
    print('-' * 20)
    print('3: previous_application (', time.ctime(), ')')
    print('-' * 20)
    prev = previous_application()
    df = df.join(prev, how = 'left', on = 'SK_ID_CURR')
    print('     DF shape:', df.shape)
    del prev
    gc.collect()
    
    print('-' * 20)
    print('4: POS_CASH_balance (', time.ctime(), ')')
    print('-' * 20)
    pos = pos_cash()
    df = df.join(pos, how = 'left', on = 'SK_ID_CURR')
    print('     DF shape:', df.shape)
    del pos
    gc.collect()
    
    print('-' * 20)
    print('5: installments_payments (', time.ctime(), ')')
    print('-' * 20)
    ins = installments_payments()
    df = df.join(ins, how = 'left', on = 'SK_ID_CURR')
    print('     DF shape:', df.shape)
    del ins
    gc.collect()
    
    print('-' * 20)
    print('6: credit_card_balance (', time.ctime(), ')')
    print('-' * 20)
    cc = credit_card_balance()
    df = df.join(cc, how = 'left', on = 'SK_ID_CURR')
    print('     DF shape:', df.shape)
    del cc
    gc.collect()
    
    print('-' * 20)
    print('7: final dataset (', time.ctime(), ')')
    print('-' * 20)
    return reduce_mem_usage(df)

## Cleaning dataset

In [46]:
def corr_feature_with_target(feature, target):
    """
    target : y value
    feature: the feature of the data set
    
    This function calculates the correlation between feature and target, and return the Wilcoxon rank-sum statistic
    as well as the 
    
    """
    c0 = feature[target == 0].dropna()
    c1 = feature[target == 1].dropna()
    
    #For binary features, calculate mean difference
    #For continuous features, calculate median difference
    if set(feature.unique()) == set([0, 1]):
        diff = abs(c0.mean(axis = 0) - c1.mean(axis = 0))
    else:
        diff = abs(c0.median(axis = 0) - c1.median(axis = 0))
    #When sample size >= 20, use Wilcoxon rank-sum statistic
    #otherwise set it to be 2
    p = ranksums(c0, c1)[1] if ((len(c0) >= 20) & (len(c1) >= 20)) else 2
        
    return [diff, p]

In [47]:
def clean_data(data):
    """
    Clean the data and return it
    """
    warnings.simplefilter(action = 'ignore')
    
    # Removing empty features
    nun = data.nunique()
    empty = list(nun[nun <= 1].index)
    
    data.drop(empty, axis = 1, inplace = True)
    print('After removing empty features there are {0:d} features'.format(data.shape[1]))
    
    # Removing features with the same distribution on 0 and 1 classes
    corr = pd.DataFrame(index = ['diff', 'p'])
    ind = data[data['TARGET'].notnull()].index
    
    for c in data.columns.drop('TARGET'):
        corr[c] = corr_feature_with_target(data.loc[ind, c].astype("float64"), data.loc[ind, 'TARGET'].astype("float64"))
    
    corr = corr.T
    gc.collect()
    corr['diff_norm'] = abs(corr['diff'] / data.mean(axis = 0))
    #if correlation is not signifcant, delete it
    to_del_1 = corr[((corr['diff'] == 0) & (corr['p'] > .05))].index
    to_del_2 = corr[((corr['diff_norm'] < .5) & (corr['p'] > .05))].drop(to_del_1).index
    to_del = list(to_del_1) + list(to_del_2)
    if 'SK_ID_CURR' in to_del:
        to_del.remove('SK_ID_CURR')
        
    data.drop(to_del, axis = 1, inplace = True)
    print('After removing features with the same distribution on 0 and 1 classes there are {0:d} features'.format(data.shape[1]))
    
    # Removing features with not the same distribution on train and test datasets
    corr_test = pd.DataFrame(index = ['diff', 'p'])
    target = data['TARGET'].notnull().astype(int)
    
    for c in data.columns.drop('TARGET'):
        corr_test[c] = corr_feature_with_target(data[c].astype("float64"), target.astype("float64"))

    corr_test = corr_test.T
    gc.collect()
    corr_test['diff_norm'] = abs(corr_test['diff'] / data.mean(axis = 0))
    
    bad_features = corr_test[((corr_test['p'] < .05) & (corr_test['diff_norm'] > 1))].index
    bad_features = corr.loc[bad_features][corr['diff_norm'] == 0].index
    
    data.drop(bad_features, axis = 1, inplace = True)
    print('After removing features with not the same distribution on train and test datasets there are {0:d} features'.format(data.shape[1]))
    
    del corr, corr_test
    gc.collect()
    
    # Removing features not interesting for classifier
    clf = LGBMClassifier(random_state = 0)
    train_index = data[data['TARGET'].notnull()].index
    train_columns = data.drop('TARGET', axis = 1).columns

    score = 1
    new_columns = []
    while score > .7:
        train_columns = train_columns.drop(new_columns)
        clf.fit(data.loc[train_index, train_columns], data.loc[train_index, 'TARGET'])
        f_imp = pd.Series(clf.feature_importances_, index = train_columns)
        score = roc_auc_score(data.loc[train_index, 'TARGET'], 
                              clf.predict_proba(data.loc[train_index, train_columns])[:, 1])
        new_columns = f_imp[f_imp > 0].index

    data.drop(train_columns, axis = 1, inplace = True)
    print('After removing features not interesting for classifier there are {0:d} features'.format(data.shape[1]))

    return data

## Optimization LGBM parameters

### Optimization and visualisation functions

In [48]:
def cv_scores(df, num_folds, params, stratified = False, verbose = -1, 
              save_train_prediction = True, train_prediction_file_name = 'train_prediction1.csv',
              save_test_prediction = True, test_prediction_file_name = 'test_prediction1.csv'):
    """
    df : the final dataset after aggregation and cleaning
    num_folds: number of folds for cross validation
    params: parameters for the LightGBM classifier
    stratified : whether to use stratified sampling or not
    verbose : whether to print model message or not while fitting model
    save_train_prediction : whether to save the prediction on training data or not
    train_prediction_file_name : the name of the file that stores the prediction on training set
    save_test_prediction : whether to save the prediction on the test dataset or not
    test_prediction_file_name : the name of the file that stores the prediction on the test dataset
    
    The function stores the final result and return the score on both training and testing dataset
    """
    
    
    warnings.simplefilter('ignore')
    
    clf = LGBMClassifier(**params)

    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits = num_folds, shuffle = True, random_state = 1001)
    else:
        folds = KFold(n_splits = num_folds, shuffle = True, random_state = 1001)
        
    # Create arrays and dataframes to store results
    train_pred = np.zeros(train_df.shape[0])
    train_pred_proba = np.zeros(train_df.shape[0])

    test_pred = np.zeros(train_df.shape[0])
    test_pred_proba = np.zeros(train_df.shape[0])
    
    prediction = np.zeros(test_df.shape[0])
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    df_feature_importance = pd.DataFrame(index = feats)
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        print('Fold', n_fold, 'started at', time.ctime())
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = verbose, early_stopping_rounds = 200)

        train_pred[train_idx] = clf.predict(train_x, num_iteration = clf.best_iteration_)
        train_pred_proba[train_idx] = clf.predict_proba(train_x, num_iteration = clf.best_iteration_)[:, 1]
        test_pred[valid_idx] = clf.predict(valid_x, num_iteration = clf.best_iteration_)
        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        prediction += \
                clf.predict_proba(test_df[feats], num_iteration = clf.best_iteration_)[:, 1] / folds.n_splits

        df_feature_importance[n_fold] = pd.Series(clf.feature_importances_, index = feats)
        
        print('Fold %2d AUC : %.6f' % (n_fold, roc_auc_score(valid_y, test_pred_proba[valid_idx])))
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    roc_auc_train = roc_auc_score(train_df['TARGET'], train_pred_proba)
    precision_train = precision_score(train_df['TARGET'], train_pred, average = None)
    recall_train = recall_score(train_df['TARGET'], train_pred, average = None)
    
    roc_auc_test = roc_auc_score(train_df['TARGET'], test_pred_proba)
    precision_test = precision_score(train_df['TARGET'], test_pred, average = None)
    recall_test = recall_score(train_df['TARGET'], test_pred, average = None)

    print('Full AUC score %.6f' % roc_auc_test)
    
    df_feature_importance.fillna(0, inplace = True)
    df_feature_importance['mean'] = df_feature_importance.mean(axis = 1)
    
    # Write prediction files
    if save_train_prediction:
        df_prediction = train_df[['SK_ID_CURR', 'TARGET']]
        df_prediction['Prediction'] = test_pred_proba
        df_prediction.to_csv(train_prediction_file_name, index = False)
        del df_prediction
        gc.collect()

    if save_test_prediction:
        df_prediction = test_df[['SK_ID_CURR']]
        df_prediction['TARGET'] = prediction
        df_prediction.to_csv(test_prediction_file_name, index = False)
        del df_prediction
        gc.collect()
    
    return df_feature_importance, \
           [roc_auc_train, roc_auc_test,
            precision_train[0], precision_test[0], precision_train[1], precision_test[1],
            recall_train[0], recall_test[0], recall_train[1], recall_test[1], 0]

In [49]:
def display_folds_importances(feature_importance_df_, n_folds = 5):
    n_columns = 3
    n_rows = (n_folds + 1) // n_columns
    _, axes = plt.subplots(n_rows, n_columns, figsize=(8 * n_columns, 8 * n_rows))
    for i in range(n_folds):
        sns.barplot(x = i, y = 'index', data = feature_importance_df_.reset_index().sort_values(i, ascending = False).head(20), 
                    ax = axes[i // n_columns, i % n_columns])
    sns.barplot(x = 'mean', y = 'index', data = feature_importance_df_.reset_index().sort_values('mean', ascending = False).head(20), 
                    ax = axes[n_rows - 1, n_columns - 1])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.show()

In [50]:
# warninig: parameters are not fully optimized due to the limit of time
# The parameter below is based on a little bit different dataset
# Hyper parameters are obtained using Bayesian-Optimization
lgbm_params = {
            'nthread': 4,
            'n_estimators': 10000,
            'learning_rate': 0.02,
            'num_leaves': 33,
            'colsample_bytree': 0.805415337217056,
            'subsample': 0.8082978617542438,
            'max_depth': 8,
            'reg_alpha': 0.04988371263522236,
            'reg_lambda': 0.0793274116877217,
            'min_split_gain':0.02817555628928521,
            'min_child_weight': 39.64402829035144,
            'silent': -1,
            'verbose': -1
}

In [51]:
#Aggregate all dataset together
full_data = aggregate()

--------------------
1: application train & test ( Wed Aug 29 12:43:18 2018 )
--------------------
The dataset application_train has the shape (307511, 122)
Memory usage of dataframe: 325.13 MB
Memory usage after optimization: 85.27 MB
Decreased by 73.8%
     DF shape: (356244, 130)
--------------------
2: bureau & balance ( Wed Aug 29 12:43:35 2018 )
--------------------
The dataset bureau_balance has the shape (27299925, 3)
The dataset bureau has the shape (1716428, 17)
Memory usage of dataframe: 477.71 MB
Memory usage after optimization: 247.02 MB
Decreased by 48.3%
     DF shape: (356244, 383)
--------------------
3: previous_application ( Wed Aug 29 12:44:30 2018 )
--------------------
Memory usage of dataframe: 471.48 MB
Memory usage after optimization: 309.01 MB
Decreased by 34.5%
The dataset previous_application has the shape (1670214, 37)
Memory usage of dataframe: 1591.24 MB
Memory usage after optimization: 991.78 MB
Decreased by 37.7%
     DF shape: (356244, 1330)
----------

In [52]:
#clean the dataset
full_data = clean_data(full_data)

After removing empty features there are 2047 features
After removing features with the same distribution on 0 and 1 classes there are 1613 features
After removing features with not the same distribution on train and test datasets there are 1612 features
After removing features not interesting for classifier there are 1574 features


In [53]:
#run garbage collector
gc.collect()

288

In [55]:
"""
def full_clf_pred(df):
  train_df = df[df['TARGET'].notnull()]
  test_df = df[df['TARGET'].isnull()]
  full_clf = LGBMClassifier(**lgbm_params)
  feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
  print("start to train")
  full_clf.fit(train_df[feats], train_df["TARGET"])
  prediction = full_clf.predict_proba(train_df[feats])[:,1]
  print("AUC of training data is {}".format(roc_auc_score(train_df["TARGET"], prediction)))
  test_prediction = full_clf.predict_proba(test_df[feats])[:,1]
  del train_df, test_df
  gc.collect()
  return(test_prediction)
  """

In [33]:
#use lightgbm classifier with 5 folds cross validation method
result = cv_scores(full_data, 5, lgbm_params)

Starting LightGBM. Train shape: (307500, 1574), test shape: (48744, 1574)
Fold 0 started at Tue Aug 28 22:24:49 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1413]	training's auc: 0.889966	valid_1's auc: 0.789092
Fold  0 AUC : 0.789061
Fold 1 started at Tue Aug 28 22:47:56 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1393]	training's auc: 0.888319	valid_1's auc: 0.79128
Fold  1 AUC : 0.791280
Fold 2 started at Tue Aug 28 23:10:01 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[2063]	training's auc: 0.915786	valid_1's auc: 0.786327
Fold  2 AUC : 0.786336
Fold 3 started at Tue Aug 28 23:37:39 2018
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[1642]	training's auc: 0.898465	valid_1's auc: 0.79132
Fold  3 AUC : 0.791320
Fold 4 started at Wed Aug 29 00:01:04 2018
Train

### Hyper parameter selection

In [62]:
#parameters obtained using bayesian optimization
lgbm_params = {'colsample_bytree': 0.805415337217056,
 'learning_rate': 0.011723218430046927,
 'max_depth': 8,
 'min_child_weight': 39.64402829035144,
 'min_split_gain': 0.02817555628928521,
 'num_leaves': 33,
 'reg_alpha': 0.04988371263522236,
 'reg_lambda': 0.0793274116877217,
 'subsample': 0.8082978617542438}

### Bayesian Optimization

In [54]:
#The function below is the LightGBM with 2 fold cross validation
#We use bayesian optimization to optimize the hyper parameters of this function
def lgbm_evaluate(**params):
    """
    **params : parameters passed to the LightGBM classifier
    
    The function returns the AUC on training dataset
    """
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
        
    clf = LGBMClassifier(**params, n_estimators = 10000, nthread = 4)

    train_df = full_data[full_data['TARGET'].notnull()]
    test_df = full_data[full_data['TARGET'].isnull()]

    folds = KFold(n_splits = 2, shuffle = True, random_state = 1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
#Initial setup of hyper parameter ranges
params = {'colsample_bytree': (0.8, 1),
          'learning_rate': (.01, .025), 
          'num_leaves': (25, 35), 
          'subsample': (0.8, 1), 
          'max_depth': (6, 9), 
          'reg_alpha': (.03, .05), 
          'reg_lambda': (.06, .08), 
          'min_split_gain': (.01, .03),
          'min_child_weight': (38, 40)}
#Bayesian optimization to select hyper parameters
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)

Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   max_depth |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 


In [61]:
#Print out the best parameters
best_params = bo.res['max']['max_params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])

best_params

{'colsample_bytree': 0.805415337217056,
 'learning_rate': 0.011723218430046927,
 'max_depth': 8,
 'min_child_weight': 39.64402829035144,
 'min_split_gain': 0.02817555628928521,
 'num_leaves': 33,
 'reg_alpha': 0.04988371263522236,
 'reg_lambda': 0.0793274116877217,
 'subsample': 0.8082978617542438}